In [156]:
import numpy as np
import cv2

class Table:
    def __init__(self, bbox = (0,0,0,0), rows = 0, cols = 0, cells = None):
        self.bbox = bbox
        self.rows = rows
        self.cols = cols
        self.cells = cells
    def disp_table(self):
        print("No. of ROWS: {}".format(self.rows))
        print("No. of COLUMNS: {}".format(self.cols))
        print("No. of Cells: {}".format(len(self.cells)))
        print("Table Width: {}".format(self.bbox[2]))
        print("Table Height: {}".format(self.bbox[3]))
        print("\n\n")

class TextBox:
    def __init__(self, bbox=(0,0,0,0)):
        self.bbox = bbox
    def disp_textbox(self):
        x,y,w,h = self.bbox
        print("x = ",x)
        print("y = ",y)
        print("w = ",w)
        print("h = ",h)
        print("\n\n")

In [176]:
img1 = cv2.imread('formexample6.png')
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img = cv2.bitwise_not(img)
img_bin,ab = cv2.threshold(img, 11, 255, cv2.THRESH_BINARY)
cv2.imwrite("RESULT2.jpg",ab)

True

In [177]:
horizontal = np.copy(ab)
# Specify size on horizontal axis
cols = horizontal.shape[1]
horizontal_size = cols // 30
# Create structure element for extracting horizontal lines through morphology operations
horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    # Apply morphology operations
horizontal = cv2.erode(horizontal, horizontalStructure)
horizontal = cv2.dilate(horizontal, horizontalStructure)
# Show extracted horizontal lines
cv2.imwrite("RESULT3.jpg",horizontal)

True

In [178]:
vertical = np.copy(ab)
# Specify size on vertical axis
rows = vertical.shape[0]
verticalsize = rows // 100
    # Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    # Apply morphology operations
vertical = cv2.erode(vertical, verticalStructure)
vertical = cv2.dilate(vertical, verticalStructure)
ch = vertical | horizontal
ch = cv2.bitwise_not(ch)
cv2.imwrite("RESULT3.jpg",ch)

True

In [179]:
contours, hierarchy = cv2.findContours(ch, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img2 = np.copy(img1)
count = 0
boxes = list()
for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w < img2.shape[1] - 60  and w>15 and h>15 and h<img2.shape[0]:
          cv2.rectangle(img2, (x, y), (x+w, y+h), (0, 0, 255), 1)
          count+=1
          boxes.append((x,y,w,h))
print(count)
print(max(boxes, key = lambda x:x[3]))
boxes.sort(key = lambda x: x[3], reverse = True)
print(boxes)
cv2.imwrite("RESULT4.jpg",img2)
cv2.imshow("IMAGE",img2)
cv2.waitKey(0)

26
(33, 199, 313, 33)
[(33, 199, 313, 33), (33, 1242, 1066, 31), (439, 932, 660, 31), (261, 932, 165, 31), (33, 932, 214, 31), (558, 571, 433, 31), (79, 571, 436, 31), (311, 489, 678, 31), (278, 431, 711, 31), (33, 142, 1068, 31), (566, 1047, 345, 29), (35, 1047, 378, 29), (450, 255, 124, 29), (377, 255, 61, 29), (273, 255, 92, 29), (724, 202, 124, 29), (566, 990, 408, 28), (35, 990, 346, 28), (442, 843, 409, 28), (35, 843, 346, 28), (653, 203, 61, 28), (583, 203, 60, 28), (48, 34, 40, 25), (1077, 614, 22, 21), (944, 1368, 136, 18), (899, 1315, 184, 17)]


-1

In [180]:
def form_tables(boxes):
    k = 0
    i = 0
    tables = list()
    tot = len(boxes)
    while i < len(boxes):
        x,y,w,h = boxes[i][0:4]
        k += 1
        j = k
        temp = list()
        while j < len(boxes):
            if boxes[j][0] <= x+w  and boxes[j][1] <= y+h and x < boxes[j][0]  and y < boxes[j][1]  :
                temp.append(boxes[j])
                boxes.remove(boxes[j])
            else:    
                j += 1
        if len(temp) == 0:
            i += 1
            continue
        temp.sort(key = lambda z: z[0])
        #print(temp)
        cols = 1
        for u in range(len(temp)-1):
            if abs(temp[u][0] - temp[u+1][0]) <= 3:
                pass
            else:
                cols += 1
        temp.sort(key = lambda z: z[1])
        rows = 1
        for u in range(len(temp)-1):
            if abs(temp[u][1] - temp[u+1][1]) <= 3:
                pass
            else:
                rows += 1
        t = Table((x,y,w,h), rows, cols, temp)
        tables.append(t)
        i += 1
    return tables, boxes
            

#tables stores objects of Table class

temp_boxes  = boxes.copy()
tables, temp_boxes = form_tables(temp_boxes)
for i in range(len(tables)):
    print("TABLE ", i+1)
    tables[i].disp_table()
        

In [181]:
for i in range(len(tables)):
    if tables[i].bbox in temp_boxes:
        temp_boxes.remove(tables[i].bbox)
TEXT_BOXES = temp_boxes.copy()
print(TEXT_BOXES)
#TEXT_BOXS stores text boxes

[(33, 199, 313, 33), (33, 1242, 1066, 31), (439, 932, 660, 31), (261, 932, 165, 31), (33, 932, 214, 31), (558, 571, 433, 31), (79, 571, 436, 31), (311, 489, 678, 31), (278, 431, 711, 31), (33, 142, 1068, 31), (566, 1047, 345, 29), (35, 1047, 378, 29), (450, 255, 124, 29), (377, 255, 61, 29), (273, 255, 92, 29), (724, 202, 124, 29), (566, 990, 408, 28), (35, 990, 346, 28), (442, 843, 409, 28), (35, 843, 346, 28), (653, 203, 61, 28), (583, 203, 60, 28), (48, 34, 40, 25), (1077, 614, 22, 21), (944, 1368, 136, 18), (899, 1315, 184, 17)]


In [182]:
#text boxes
img3 = img2
for i in TEXT_BOXES:
    x,y,w,h = i[0:4]
    cv2.rectangle(img3, (x, y), (x+w, y+h), (255, 0, 0), 3)
cv2.imwrite("RESULT4.jpg", img3)

True

In [183]:
text_boxes = [TextBox(x) for x in TEXT_BOXES]
for i in range(len(text_boxes)):
    text_boxes[i].disp_textbox()

x =  33
y =  199
w =  313
h =  33



x =  33
y =  1242
w =  1066
h =  31



x =  439
y =  932
w =  660
h =  31



x =  261
y =  932
w =  165
h =  31



x =  33
y =  932
w =  214
h =  31



x =  558
y =  571
w =  433
h =  31



x =  79
y =  571
w =  436
h =  31



x =  311
y =  489
w =  678
h =  31



x =  278
y =  431
w =  711
h =  31



x =  33
y =  142
w =  1068
h =  31



x =  566
y =  1047
w =  345
h =  29



x =  35
y =  1047
w =  378
h =  29



x =  450
y =  255
w =  124
h =  29



x =  377
y =  255
w =  61
h =  29



x =  273
y =  255
w =  92
h =  29



x =  724
y =  202
w =  124
h =  29



x =  566
y =  990
w =  408
h =  28



x =  35
y =  990
w =  346
h =  28



x =  442
y =  843
w =  409
h =  28



x =  35
y =  843
w =  346
h =  28



x =  653
y =  203
w =  61
h =  28



x =  583
y =  203
w =  60
h =  28



x =  48
y =  34
w =  40
h =  25



x =  1077
y =  614
w =  22
h =  21



x =  944
y =  1368
w =  136
h =  18



x =  899
y =  1315
w =  184
h =  17



